In [1]:
import pandas as pd
import numpy as np

목표
1. 하나 이상의 키를 사용해 pandas 객체를 여러 조각으로 나누기
2. 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
3. 정규화, 선형회귀, 등급 또는 부분집합 선택 갗은 집단 내 변형이나 다른 조작을 적용하는 방법
4. 피벗테이블과 교차일람표 구하는 방법
5. 변위치 분석과 다른 통계 집단 분석을 수행하는 방법

# GroupBy 메카닉
그룹 연산
- (분리) pandas객체나 다른 객체에 들어있는 데이터를 하나 이상의 키를 기준으로 분리
- (적용) 함수를 각 그룹에 적용시켜 새로운 값을 얻어냄
- (결합) 함수를 적용한 결과를 하나의 객체로 결합

In [2]:
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                  'key2': ['one','two','one','two','one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.870342,0.239068
1,a,two,0.955398,1.275585
2,b,one,0.842060,-1.439075
3,b,two,-0.340480,0.129271
4,a,one,0.164396,-0.726185


In [3]:
# 위 데이터를 key1으로 묶고 각 그룹에서 data1의 평균 구하기

# groupby 메서드를 호출하고 key1 컬럼을 넘김
grouped = df['data1'].groupby(df['key1'])

grouped       # key1 컬럼에 있는 유일한 값으로 색인되는 Series객체가 생성됨, 생성된 객체의 색인은 key1

In [4]:
grouped.mean()

key1
a   -0.250182
b    0.250790
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()  # 여러 개의 배열을 '리스트' 넘겨 계층적 색인 삼음
means

key1  key2
a     one    -0.852973
      two     0.955398
b     one     0.842060
      two    -0.340480
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.852973,0.955398
b,0.842060,-0.340480


In [7]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])

# 길이만 같다면 어떤 배열이라도 상관 없음, 이 경우는 key1, key2가 states,years로 바뀐 채(같은 색인) 평균 구하는 연산 진행된 것
df['data1'].groupby([states,years]).mean()

California  2005    0.955398
            2006    0.842060
Ohio        2005   -1.105411
            2006    0.164396
Name: data1, dtype: float64

### DataFrame에 대하여

In [8]:
df.groupby('key1').mean()

# 출력물을 보면 컬럼에 key2가 빠져있다. 숫자데이터가 아니므로 '성가신 컬럼' 취급해 자동으로 결과에서 제외한 것

,data1,data2
key1,,
a,-0.250182,0.262823
b,0.250790,-0.654902


In [9]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.852973 -0.243558
     two   0.955398  1.275585
b    one   0.842060 -1.439075
     two  -0.340480  0.129271

### size 메서드 : 그룹의 크기를 담고 있는 Series 반환 

In [10]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 그룹간 순회

In [11]:
# GroupBy 객체는 이터레이션을 지원, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환

for name, group in df.groupby('key1'):
    print(name)   # a와 b
    print('\n')
    print(group)  #아래 DataFrame들(type을 검사해보면 DataFrame임)
    print('\n')

a


  key1 key2     data1     data2
0    a  one -1.870342  0.239068
1    a  two  0.955398  1.275585
4    a  one  0.164396 -0.726185


b


  key1 key2    data1     data2
2    b  one  0.84206 -1.439075
3    b  two -0.34048  0.129271




In [12]:
# 기준을 두 개로 주어서 묶는 것도 마찬가지 

for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.870342  0.239068
4    a  one  0.164396 -0.726185
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.955398  1.275585
('b', 'one')
  key1 key2    data1     data2
2    b  one  0.84206 -1.439075
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.34048  0.129271


In [13]:
# 기본적으로 axis = 0에 대하여 그룹을 만들지만 다른 축으로 하는 것도 가능

df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [14]:
# 다음은 df.dtypes를 기준으로 삼아 그룹을 묶음(data1,data2가 하나로, key1,key2가 하나로 묶임)

grouped = df.groupby(df.dtypes, axis=1)
for dtype,group in grouped:
    print(dtype)
    print('\n')
    print(group)
    print('\n')

float64


      data1     data2
0 -1.870342  0.239068
1  0.955398  1.275585
2  0.842060 -1.439075
3 -0.340480  0.129271
4  0.164396 -0.726185


object


  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one




### 원하는 데이터만 골라내기

In [15]:
# 그룹별 데이터를 사전형으로 바꿔서 사용하기

pieces = dict(list(df.groupby('key1')))

# pieces는 key를 a,b로, value를 DataFrame으로 갖는 dictionary

pieces['a']

,key1,key2,data1,data2
0,a,one,-1.870342,0.239068
1,a,two,0.955398,1.275585
4,a,one,0.164396,-0.726185


## 컬럼이나 컬럼의 일부만 선택하기
df.groupby('key1')['data1'] = df['data1'].groupby(df['key1'])

df.groupby('key1')[['data2']] = df[['data2']].groupby(df['key1'])

각각 앞의 것들은 뒤의 코드에 대한 신택틱 슈거로 같은 결과 반환

In [16]:
# data2 컬럼의 자료를 key1, key2를 기준으로 분류

df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.243558
     two   1.275585
b    one  -1.439075
     two   0.129271

In [17]:
# 색인으로 얻은 객체는 groupby 메서드에 '리스트나 배열'을 넘겼을 경우 DataFrameGroupBy 객체가 되고
# '단일 값'으로 하나의 컬럼 이름만 넘겼을 경우 SeriesGroupBy 객체가 된다.

df.groupby(['key1','key2'])['data2'].mean() 

key1  key2
a     one    -0.243558
      two     1.275585
b     one    -1.439075
      two     0.129271
Name: data2, dtype: float64

## 사전과 Series에서 그룹핑하기

In [18]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns=['a','b','c','d','e'],index=['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2:3,[1,2]] = np.nan   # NaN 값을 추가합시다.
people

,a,b,c,d,e
Joe,-1.786102,-0.590097,1.270454,0.054005,-1.133983
Steve,0.844258,0.616260,0.411843,-0.518793,-0.351568
Wes,-0.993819,NaN,NaN,1.047129,-0.566885
Jim,-1.554507,-0.216260,-0.046433,0.002330,0.416481
Travis,0.547346,-2.070112,-0.609910,-0.033781,-0.553264


In [19]:
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

by_column = people.groupby(mapping,axis=1)   # red(a,b,e)끼리, blue(c,d)끼리 묶음
by_column.sum()

,blue,red
Joe,1.324459,-3.510181
Steve,-0.106950,1.108949
Wes,1.047129,-1.560704
Jim,-0.044103,-1.354287
Travis,-0.643691,-2.076030


In [20]:
# 딕셔너리 대신 Series에 대해서도 같은 기능을 수행할 수 있다.

map_series = pd.Series(mapping)
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,1.324459,-3.510181
Steve,-0.106950,1.108949
Wes,1.047129,-1.560704
Jim,-0.044103,-1.354287
Travis,-0.643691,-2.076030


## 함수로 그룹핑

In [21]:
# len은 사람 이름에 대한 길이를 기준으로 분류함

people.groupby(len).sum()

,a,b,c,d,e
3,-4.334428,-0.806357,1.224021,1.103464,-1.284387
5,0.844258,0.616260,0.411843,-0.518793,-0.351568
6,0.547346,-2.070112,-0.609910,-0.033781,-0.553264


In [22]:
# 내부적으로는 모두 배열로 변환되므로 함수를 배열, 사전, Series와 섞어 쓰더라고 전혀 문제되지 않는다

key_list = ['one','one','one','two','two']     # len이 3인 사람 중 Joe, Wes는 one, Jim은 two
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -1.786102 -0.590097  1.270454  0.054005 -1.133983
  two -1.554507 -0.216260 -0.046433  0.002330  0.416481
5 one  0.844258  0.616260  0.411843 -0.518793 -0.351568
6 two  0.547346 -2.070112 -0.609910 -0.033781 -0.553264

## 색인 단계로 그룹핑하기 (다중 색인에서)

In [23]:
# 계층적으로 색인된 데이터는 축 색인의 단계 중 하나를 사용해서 편리하게 집계할 수 있는 기능을 제공한다.

columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])
# ['US','US','US','JP','JP'] 컬럼의 이름은 cty
# [1,3,5,1,3] 컬럼의 이름은 tenor

hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      2.271515 -0.569129  0.816498  0.044272 -1.085990
1     -0.999420  1.904855  0.713697 -1.201491  0.564283
2     -1.022291 -0.374638 -0.509036 -0.458544  0.696017
3      0.490015  0.109211  1.404404  2.059306  0.103270

In [24]:
# 색인 중 하나를 사용하는 기능을 사용하고자 한다면 level 예약어를 사용해서 레벨 번호나 이름을 넘기면 된다.

hier_df.groupby(level='tenor',axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


# 데이터 집계
데이터 집계는 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업을 말한다

해당 메서드
- count, sum, mean, median, min, max, prod(곱), first, last, std, var

위 메서드들은 결측치(NaN)가 아닌 값들만을 대상으로 연산을 수행한다

직접 고안한 집계함수를 사용하고 추가적으로 그룹 객체에 이미 정의된 메서드를 연결해서 사용하는 것 역시 가능

In [25]:
df

,key1,key2,data1,data2
0,a,one,-1.870342,0.239068
1,a,two,0.955398,1.275585
2,b,one,0.842060,-1.439075
3,b,two,-0.340480,0.129271
4,a,one,0.164396,-0.726185


In [26]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)    # 변위치 계산

key1
a    0.797198
b    0.723806
Name: data1, dtype: float64

참고, a: array, q: array or float
- np.quantile(a,q) -> 배열 a에서의 q에 해당하는 분위수를 계산

### 사용자 정의 함수 사용

In [27]:
# agg 혹은 aggregate 메서드에 해당 함수를 넘김

def peakpeak(arr):
    return arr.max()-arr.min()
grouped.agg(peakpeak)

# 단, 사용자 정의 함수는 일반적으로 정의된 함수에 비해 매우 느리게 동작한다

,data1,data2
key1,,
a,2.82574,2.001770
b,1.18254,1.568346


## 컬럼에 여러 가지 함수 적용하기

In [28]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip']/tips['total_bill']  # 새 컬럼 추가
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [29]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']

# 이미 통용되는 기술 통계 함수는 함수 이름을 문자열('')로 넘기면 된다.

grouped_pct.agg('mean')
# 일단은 함수 하나이므로 grouped_pct.mean()와 결과가 같다.

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [30]:
grouped_pct.agg(['mean','std',peakpeak])  # 함수목록을 리스트로 넘기면 DataFrame을 얻을 수 있다.
                                          # 사용자 정의 함수는 ''를 붙히지 않는다.

mean       std  peakpeak
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [31]:
# 함수 이름이 아닌 다른 이름으로 컬럼 이름을 지정하는 방법

grouped_pct.agg([('평균','mean'),('표준편차',np.std)])

평균      표준편차
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [32]:
# DataFrame은 컬럼마다 다른 함수를 적용하거나 여러 개의 함수를 모든 컬럼에 적용할 수 있다.

functions = [('개수','count'),'mean','max']
result = grouped['tip_pct','total_bill'].agg(functions)

# 다중 색인된 컬럼을 가진 DataFrame 반환
result

tip_pct                     total_bill                  
                 개수      mean       max         개수       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [33]:
# 해당 DataFrame은 계층적 컬럼을 가지고 있음. 
# 각 컬럼을 따로 계산한 다음 concat 메서드를 이용해 keys인자로 컬럼이름을 넘겨서 이어붙인 것과 동일
# pd.concat([result['tip_pct'],result['total_bill']],axis=1)
result['tip_pct']

개수      mean       max
day  smoker                        
Fri  No       4  0.151650  0.187735
     Yes     15  0.174783  0.263480
Sat  No      45  0.158048  0.291990
     Yes     42  0.147906  0.325733
Sun  No      57  0.160113  0.252672
     Yes     19  0.187250  0.710345
Thur No      45  0.160298  0.266312
     Yes     17  0.163863  0.241255

In [34]:
# 컬럼마다 다른 함수를 사용하고 싶다면 딕셔너리를 이용

grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [35]:
# 단 하나의 컬럼에라도 여러 개의 함수가 적용되었다면 DataFrame은 계층적 컬럼을 가지게 된다.

grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [36]:
tips.groupby(['day','smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [37]:
# as_index=False를 넘겨 색인되지 않도록 할 수 있다.(기준이 되는 day,smoker컬럼까지 컬럼으로 반환)
tips.groupby(['day','smoker'],as_index=False).mean()

# 혹은 색인된 결과에 reset_index 메서드를 호출해 같은 결과를 얻을 수 있다.
# tips.groupby(['day','smoker']).mean().reset_index()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


참고) reset_index : 계층적 색인 단계가 컬럼으로 이동

# Apply : 일반적인 분리 - 적용 - 병합
apply 메서드는 객체를 여러 조각으로 나누고, 전달된 함수를 각 조각에 일괄 적용한 후 이를 다시 합친다.

In [38]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]       
# 특정 컬럼(기본값은 tip_pct)에서 가장 큰 값을 가지는 로우를 선택하는 함수

top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [39]:
tips.groupby('smoker').apply(top)

# 흡연 여부를 No와 Yes로 분류한 상태로 top을 적용
# --> 나누어진 DataFrame의 각 부분에 모두 적용 

# --> pd.concat으로 합쳐진 후 이름 붙은 꼴

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [50]:
# apply 메서드로 넘길 함수가 추가적인 인자를 받는다면 함수 이름 뒤에 붙여서 넘겨주면 된다.

tips.groupby(['smoker','day']).apply(top,n=2,column='total_bill')

# 두 개의 분류 기준값마다 두 개(n=2)씩이 최대값들이 출력됨. 최대값의 기준 컬럼은 totall_bill

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

## 그룹 색인 생략하기

In [43]:
# group_keys=False 옵션을 통해 색인 생략 가능
# as_index=False는 0,1,2 등 정수 색인이 붙지만 grop_keys=False는 특별한 색인이 추가되지 않는다.

tips.groupby('smoker',group_keys=False).apply(top)   # 아래 셀과 비교

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [46]:
tips.groupby('smoker',as_index=False).apply(top)

total_bill   tip smoker   day    time  size   tip_pct
0 88        24.71  5.85     No  Thur   Lunch     2  0.236746
  185       20.69  5.00     No   Sun  Dinner     5  0.241663
  51        10.29  2.60     No   Sun  Dinner     2  0.252672
  149        7.51  2.00     No  Thur   Lunch     2  0.266312
  232       11.61  3.39     No   Sat  Dinner     2  0.291990
1 109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
  183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
  67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
  178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
  172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

## 변위치 분석과 버킷 분석
- cut(~,4)는 그룹의 크기를 4등분하는 구간 (처음값과 끝값이 중요)
    - list1, list2(list1 보다 1만큼 작은 리스트)를 옵션으로 주는 것도 가능
- qcut(~,4)는 분위수에 맞게 곧 구간마다 원소들의 개수가 일정하게 하는 4개 구간으로 나눔
   
- precision =2 하면 소수점 2자리까지로 제한

In [96]:
frame = pd.DataFrame({'data1':np.random.randn(1000),'data2':np.random.randn(1000)})
quartiles = pd.cut(frame.data1,4)

quartiles[:10]

0    (-1.508, 0.00681]
1    (-1.508, 0.00681]
2     (0.00681, 1.522]
3    (-1.508, 0.00681]
4    (-1.508, 0.00681]
5    (-1.508, 0.00681]
6     (-3.029, -1.508]
7       (1.522, 3.036]
8    (-1.508, 0.00681]
9    (-1.508, 0.00681]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.029, -1.508] < (-1.508, 0.00681] < (0.00681, 1.522] < (1.522, 3.036]]

In [92]:
# data2 컬럼에 대한 몇 가지 통계 계산

def get_starts(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}

frame.data2.groupby(quartiles).apply(get_starts).unstack()
# 인덱스 명은 위에서 data1으로 설정되어있음 (컬럼마다의 계산값은 data2를 나타내지만 분류기준은 data1임)

,min,max,count,mean
data1,,,,
"(-3.103, -1.583]",-1.490620,2.134678,48.0,0.220166
"(-1.583, -0.0683]",-2.866741,2.918839,428.0,-0.042740
"(-0.0683, 1.446]",-3.370874,3.460071,458.0,0.015167
"(1.446, 2.96]",-3.155072,2.408551,66.0,0.037535


In [97]:
# 위는 등간격 버킷, 아래는 표본 변위치에 기반한 버킷

# labels=False로서 각 색인에 해당하는 값들이 몇 번째 구간에 속해있나만 나타냄

grouping = pd.qcut(frame.data1,10,labels=False)
grouping   

0      2
1      0
2      6
3      2
4      3
      ..
995    2
996    9
997    8
998    8
999    3
Name: data1, Length: 1000, dtype: int64

In [102]:
# 마찬가지로 각 data1 의 구간에 따른 data2 값들의 (함수가 적용된 형태로) 반환

frame.data2.groupby(grouping).apply(get_starts).unstack()

,min,max,count,mean
data1,,,,
0,-2.628588,2.297374,100.0,0.071435
1,-2.499010,1.813246,100.0,-0.044250
2,-2.792835,2.546726,100.0,-0.099375
3,-3.118551,2.856035,100.0,-0.045639
4,-1.748733,1.684735,100.0,0.054735
5,-2.127552,2.578335,100.0,-0.015894
6,-2.276481,2.486497,100.0,0.039979
7,-2.617622,2.322578,100.0,-0.022053
8,-2.937915,2.484183,100.0,0.006038


## 예제 : 그룹에 다른 값으로 결측치 채우기
(dropna 대신 fillna 메서드로 결측치 대체할 것)

In [103]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.883115
2         NaN
3   -1.923475
4         NaN
5    0.265202
dtype: float64

In [104]:
# s.mean()은 s 전체의 mean

s.fillna(s.mean())

0   -0.258386
1    0.883115
2   -0.258386
3   -1.923475
4   -0.258386
5    0.265202
dtype: float64

In [105]:
# 그룹별로 채워넣고 싶은 값이 다르다면 데이터를 그룹으로 나누고 apply함수를 사용해서 각 그룹에 대한 fillna를 적용하면 된다.

states=['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']
data = pd.Series(np.random.randn(8),index=states)
data[['Vermont','Nevada','Idaho']] = np.nan
data

Ohio          0.072428
New York     -0.397684
Vermont            NaN
Florida       0.119908
Oregon        0.584075
Nevada             NaN
California    0.574923
Idaho              NaN
dtype: float64

In [106]:
group_key = ['East']*4 + ['West']*4
data.groupby(group_key).mean()

East   -0.068449
West    0.579499
dtype: float64

In [107]:
# 결측치가 위아래 로우의 평균값으로 채워진다.

fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.072428
New York     -0.397684
Vermont      -0.068449
Florida       0.119908
Oregon        0.584075
Nevada        0.579499
California    0.574923
Idaho         0.579499
dtype: float64

In [108]:
fill_values = {'East':0.5,'West':-1}

# 각 그룹은 내부적으로 name이라는 속성 가지고 있음

fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.072428
New York     -0.397684
Vermont       0.500000
Florida       0.119908
Oregon        0.584075
Nevada       -1.000000
California    0.574923
Idaho        -1.000000
dtype: float64

## 예제 : 랜덤 표본과 순열
대용량의 데이터를 몬데카를로 시뮬레이션이나 다른 애플리케이션에서 사용하기 위해 랜덤 표본을 뽑아낸다고 하자.

In [109]:
# 하트, 스페이드, 클럽, 다이아몬드
suits = ['H','S','C','D']
card_val = (list(range(1,11))+[10]*3)*4
base_names=['A']+list(range(2,11))+['J','K','Q']
cards=[]
for suit in suits:
    cards.extend(str(num)+suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck[:15]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
dtype: int64

In [114]:
# 다섯 장의 카드 뽑기

def draw(deck,n=5):
    return deck.sample(n)

draw(deck)

9C      9
10S    10
4D      4
8D      8
8C      8
dtype: int64

In [54]:
# 각 세트(하트,스페이드,클럽,다이아몬드)별로 2장의 카드를 무작위로 뽑기

# 마지막 글자가 세트
get_suit = lambda card: card[-1]

deck.groupby(get_suit).apply(draw,n=2)
# groupby의 괄호안에는 함수가 들어가도 상관없음. 내부적으로는 배열로 변환되서 계산 

C  QC     10
   3C      3
D  4D      4
   2D      2
H  JH     10
   10H    10
S  3S      3
   9S      9
dtype: int64

## 예제 : 그룹 가중 평균과 상관관계

In [115]:
df = pd.DataFrame({'category': ['a','a','a','a','b','b','b','b'],
                  'data': np.random.randn(8),
                  'weights': np.random.rand(8)})    # weights는 가중평균
df

,category,data,weights
0,a,-1.102635,0.639452
1,a,-1.318500,0.893328
2,a,0.098490,0.603638
3,a,-0.190709,0.085669
4,b,0.439887,0.839885
5,b,1.522780,0.717873
6,b,-0.840317,0.181137
7,b,1.002068,0.515113


In [116]:
grouped = df.groupby('category')

# category별 그룹 가중평균
# np.average(list,weights=weights)  list의 원소들에 weights만큼의 가중치를 부과해 그 가중평균을 구한다.

get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

grouped.apply(get_wavg)

category
a   -0.827970
b    0.810371
dtype: float64

## 퍼센트 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관관계를 살펴보도록 합시다.

In [68]:
close_px = pd.read_csv('stock_px_2.csv',parse_dates=True,index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [69]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [72]:
# 우선 SPX 컬럼과 다른 컬럼과의 상관관계 계산
spx_corr = lambda x:x.corrwith(x['SPX'])

# pct_change 함수를 이용해 close_px의 퍼센트 변화율을 계산
rets = close_px.pct_change().dropna()

# 각 datetime에서 연도속성만을 반환하는 함수를 이용해 연도별 퍼센트 변화율 구한다.
get_year = lambda x:x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [73]:
# 두 컬럼간의 상관관계 계산(다음은 애플과 마이크로소프트)

by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 예제 : 그룹상의 선형회귀
pandas 객체나 스칼라값을 반환하기만 한다면 groupby를 좀 더 복잡한 그룹상의 통계 분석을 위해 사용할 수 있다.

예를 들어 계량경제 라이브러리인 statsmodels를 사용해 regress라는 함수를 작성하고 각 데이터 묶음마다 최소제곱으로 회귀를 수행할 수 있다.

In [74]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y,X).fit()
    return result.params

In [75]:
# SPX 수익률에 대한 애플(AAPL) 주식의 연간 선형회귀

by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 피벗테이블과 교차일람표
데이터 요약화 도구. 

하나 이상의 키로 수집해 어떤 키는 로우에, 어떤 키는 컬럼에 나열해서 데이터를 정렬한다.

pandas에서 피벗테이블은 groupby 기능을 사용해 계층적 색인을 활용한 재형성 연산을 가능하게 해준다.

DataFrame에는 pivot_table 메서드가 있는데 이는 pandas 모듈의 최상위 함수로도 존재한다.

pivot_table은 마진이라고 하는 부분합을 추가할 수 있는 기능을 제공한다.

In [117]:
# tips의 여섯 개 컬럼 중 두 개를 인덱스로 사용

tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [118]:
# 나타낼 값, 기준이 될 인덱스, 컬럼을 지정

tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [119]:
# margines=True를 넘겨서 부분합을 포함하도록 확장 가능 --> All컬럼과 All로우가 추가되어 단일 줄 내에서의 그룹 통계 얻음
# aggfunc를 비워두면 margin=True로 sum 계산

# 다른 집계함수를 사용하기 위해서는 aggfunc로 넘기면 되는데 (DataFrame안에 값으로 들어가있는 것들)
# 예를 들어 'count'나 len함수는 그룹 크기의 교차일람표(총 개수나 빈도)를 반환한다.

tips.pivot_table(['tip_pct'],index=['time','smoker'],columns='day',aggfunc=len, margins=True)

tip_pct                         
day               Fri   Sat   Sun  Thur    All
time   smoker                                 
Dinner No         3.0  45.0  57.0   1.0  106.0
       Yes        9.0  42.0  19.0   NaN   70.0
Lunch  No         1.0   NaN   NaN  44.0   45.0
       Yes        6.0   NaN   NaN  17.0   23.0
All              19.0  87.0  76.0  62.0  244.0

In [123]:
# 만일 어떤 조합이 비어있다면 fill_value를 넘길 수 있음

tips.pivot_table(['tip_pct'],index=['time','size','smoker'],columns='day',aggfunc='mean', fill_value=0)

tip_pct                              
day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## 교차일람표
그룹 빈도를 계산하기 위한 피벗테이블의 특수한 경우

In [14]:
d=[list(range(1,11)),['USA','Japan','USA','Japan','Japan','Japan','USA','USA','Japan','USA'],
   ['Right','Left','Right','Right','Left','Right','Right','Left','Right','Right']]
a=np.array(d).T
data=pd.DataFrame(a,columns=['Sample','Nationality','Handedness'])
data

,Sample,Nationality,Handedness
0,1,USA,Right
1,2,Japan,Left
2,3,USA,Right
3,4,Japan,Right
4,5,Japan,Left
5,6,Japan,Right
6,7,USA,Right
7,8,USA,Left
8,9,Japan,Right
9,10,USA,Right


In [15]:
# 요약

pd.crosstab(data.Nationality,data.Handedness,margins=True)

Handedness,Left,Right,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [17]:
# crosstab함수의 처음 두 인자는 배열이나 Series 혹은 배열의 리스트가 될 수 있다.

pd.crosstab([tips.time, tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244